In [4]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import folium
from folium.plugins import HeatMap
from streamlit_folium import st_folium
import sqlite3
import time

# Carregar dados do banco (ou DataFrame 'dados')
def carregar_dados_atualizados(nome_banco, nome_tabela):
    conn = sqlite3.connect(nome_banco)
    try:
        df = pd.read_sql_query(f"SELECT * FROM {nome_tabela}", conn)
    finally:
        conn.close()
    return df

# Função para filtrar e processar os atrasos
def filtrar_e_processar_atrasos(dados):
    df_atrasados = dados.loc[dados['SITUACAO'] == 'ATRASADO'].dropna(subset=['LAT_IN_TIME', 'LON_IN_TIME']).reset_index(drop=True)
    return df_atrasados

# Função para aplicar DBSCAN e agrupar atrasos por clusters
def aplicar_dbscan_e_gerar_clusters(df_atrasados):
    coords = df_atrasados[['LAT_IN_TIME', 'LON_IN_TIME']].values
    kms_per_radian = 6371.0088
    epsilon = 0.5 / kms_per_radian
    db = DBSCAN(eps=epsilon, min_samples=5, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    df_atrasados['cluster'] = db.labels_
    df_clusters = df_atrasados[df_atrasados['cluster'] != -1]
    atrasos_por_cluster = df_clusters.groupby('cluster').agg({
        'LAT_IN_TIME': 'mean',
        'LON_IN_TIME': 'mean',
        'cluster': 'size'
    }).rename(columns={'cluster': 'count'}).reset_index()
    return atrasos_por_cluster, df_clusters

# Função para criar o mapa com clusters de atrasos
def criar_mapa_de_atrasos(atrasos_por_cluster, df_atrasados):
    latitude_media = df_atrasados['LAT_IN_TIME'].mean()
    longitude_media = df_atrasados['LON_IN_TIME'].mean()
    mapa = folium.Map(location=[latitude_media, longitude_media], zoom_start=13)
    heat_data = df_atrasados[['LAT_IN_TIME', 'LON_IN_TIME']].values.tolist()
    HeatMap(heat_data).add_to(mapa)
    for _, cluster in atrasos_por_cluster.iterrows():
        folium.Marker(
            location=[cluster['LAT_IN_TIME'], cluster['LON_IN_TIME']],
            popup=f'Cluster {cluster["cluster"]}: {cluster["count"]} atrasos',
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(mapa)
    return mapa

# Configuração do dashboard com atualização em tempo real
st.title("Dashboard de Atrasos em Tempo Real")
st.markdown("### Mapa de Clusters de Atrasos")
intervalo_atualizacao = 60  # Intervalo em segundos para atualização

while True:
    dados = carregar_dados_atualizados('dados_onibus.db', 'onibus_buffer')
    df_atrasados = filtrar_e_processar_atrasos(dados)
    atrasos_por_cluster, df_clusters = aplicar_dbscan_e_gerar_clusters(df_atrasados)
    mapa = criar_mapa_de_atrasos(atrasos_por_cluster, df_atrasados)
    st_folium(mapa, width=700, height=500)
    time.sleep(intervalo_atualizacao)


2024-10-31 10:03:53.129 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-31 10:03:53.131 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-31 10:03:53.162 
  command:

    streamlit run /home/felipe/.local/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-10-31 10:03:53.163 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-31 10:03:53.164 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-31 10:03:53.165 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-31 10:03:53.225 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored whe

KeyboardInterrupt: 